# Exploring `PyPestUtils.helpers` with a structured Freyberg model

In [ ]:
import os
import sys
import shutil
import subprocess as sp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Setup the model dir

In [ ]:
org_d = "freyberg_daily"
w_d = "freyberg_helpers"
if os.path.exists(w_d):
    shutil.rmtree(w_d)
shutil.copytree(org_d,w_d)

In [ ]:
delc = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delc.txt")).flatten()
delr = np.loadtxt(os.path.join(w_d,"freyberg6.dis_delr.txt")).flatten()

In [ ]:
nrow = delc.shape[0]
ncol = delr.shape[0]
ib = np.loadtxt(os.path.join(w_d,"freyberg6.dis_idomain_layer1.txt"),dtype=int)
ib = ib.flatten().reshape(nrow,ncol)
plt.imshow(ib)

In [ ]:
sys.path.insert(0,"..")
import pypestutils.helpers as helpers


In [ ]:
sr = helpers.SpatialReference(delc=delc,delr=delr,rotation=-55,xul=0,yul=0)
sr.rotation

In [ ]:
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)

In [ ]:
gridspec_fname = os.path.join(w_d,"grid.spc")
sr.write_gridspec(gridspec_fname)

In [ ]:
grid_info = helpers.get_grid_info_from_gridspec(gridspec_fname)
grid_info

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(10,gridspec_fname)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)
plt.scatter(ppdf.x,ppdf.y)

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(10,gridspec_fname,zone_array=ib)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,ib)
plt.scatter(ppdf.x,ppdf.y)

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10,zone_array=ib)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

In [ ]:
bearing = np.add(np.ones((nrow,ncol)),np.atleast_2d(np.arange(ncol)))
#plt.imshow(bearing)
#bearing.min(),bearing.max()

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10,zone_array=ib,variobearing=bearing,varioaniso=10,variorange=1000)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

In [ ]:
s = 10**(np.sin(np.linspace(0,np.pi*2,nrow)))
#plt.plot(s) 
aniso = np.add(np.ones((nrow,ncol)),np.atleast_2d(s).transpose())
#plt.imshow(aniso)
#aniso.min(),aniso.max()

In [ ]:
reals = helpers.generate_2d_grid_realizations(gridspec_fname,num_reals=10,zone_array=ib,variobearing=bearing,varioaniso=aniso,variorange=1000)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0,:,:])

In [ ]:
ppdf = helpers.get_2d_pp_info_structured_grid(10,gridspec_fname,zone_array=ib,existing_array=reals[0,:,:],bearing_array=bearing)
fig,axes = plt.subplots(1,2,figsize=(10,10))
axes[0].pcolormesh(sr.xcentergrid,sr.ycentergrid,reals[0])
axes[0].scatter(ppdf.x,ppdf.y,c="k")
axes[1].scatter(ppdf.x,ppdf.y,c=ppdf.value)
for ax in axes:
    ax.set_aspect("equal")


In [ ]:
ppdf.shape

In [ ]:
ppdf.loc[:,"aniso"] = 10

In [ ]:
interp = helpers.interpolate_with_sva_pilotpoints_2d(ppdf,gridspec_fname,zone_array=ib)
plt.pcolormesh(sr.xcentergrid,sr.ycentergrid,interp)